# PDF 문서를 image로 변환 후 요약하는 샘플 코드 (2)

In [1]:
!pip install PyMuPDF Pillow boto3

In [2]:
# 앞서 이미지 방식으로 추출한 내용 로딩
%store -r img_context

In [3]:
#img_context

## 1. 매서드 작성

In [4]:
import fitz  # PyMuPDF
import boto3
import json
import time
from botocore.exceptions import ClientError

In [5]:
# 1. PDF 파싱
def parse_pdf(file_path):
    doc = fitz.open(file_path)
    text = ""
    for page in doc:
        text += page.get_text()
    return text

In [6]:
# 2. Amazon Bedrock 클라이언트 설정
bedrock = boto3.client(
    service_name='bedrock-runtime',
    region_name='us-east-1'  # 예: 'us-east-1'
)

In [9]:
# 3. Claude 모델에 이미지 전송 및 응답 받기
def summarize_with_claude(text):
    
    prompt = f"""
    다음 <instruction></instructon>에 따라 <full_contents>{text}</full_contents> 문서를 요약해주세요.
    <instruction>
    1. 문서의 목차 구성은 <context>{img_context}</context> 내용을 참조하세요.
    2. 문서를 목차(Content, Agenda 등) 별로 구분하여 문서 내용을 최대한 자세히 적어주세요. 중요 정보나 객관적 데이터를 누락하지 말고 반드시 포함하세요.
    </instruction>
    """
    
    body = {
        "anthropic_version": "bedrock-2023-05-31",
        "max_tokens": 4096,
        "temperature": 0,
        "top_p": 0,
        "top_k": 0,
        "messages": [
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": prompt
                    }
                ]
            }
        ]
    }

    #테스트 환경에 Claude 3.5 사용이 불가합니다. 고객 분들은 아래 modelId를 Claude 3.5 Sonnet 으로 변경해주세요.
    response = bedrock.invoke_model(
        body=json.dumps(body),
        #modelId = "anthropic.claude-3-5-sonnet-20240620-v1:0"  # Claude 3.5 Sonnet
        modelId="anthropic.claude-3-sonnet-20240229-v1:0",      # Claude 3 Sonnet
        contentType="application/json",
        accept="application/json"
    )

    response_body = json.loads(response['body'].read())
    return response_body['content'][0]['text']

## 2. Claude 3.x 모델 호출하여 소제목 별 요약 수행

#### Claude 3.5 Sonnet 의 이미지 분석 성능이 더 뛰어납니다. 아래 결과는 Claude 3 Sonnet 결과 입니다,

In [10]:
%%time

pdf_path = "./rag_data/Steel Business Briefing_04 Jul 2024.pdf"

# 1. PDF 파싱
parsed_text = parse_pdf(pdf_path)

# 2. Claude 3 Sonnet을 이용한 요약
summary = summarize_with_claude(parsed_text)

# 3. 요약 결과 출력
print("요약 결과:")
print(summary)

요약 결과:
이 문서는 SBB Daily Briefing의 2024년 7월 4일자 보고서로, 주요 내용은 다음과 같습니다:

Price snapshot
- 다양한 철강 제품의 심볼, 단위, 가격, 변동, 변동률, 평가일자가 테이블 형식으로 제시되어 있음

Contents
Asia
Flat Products
- Asian HRC 시장 동향 정체, 구매 관심 부족
- 중국 수출 오퍼 $520/mt FOB, 바이어 입찰가 $510/mt FOB 수준
- 일본, 한국 HRC 가격 경쟁력 상승

Indian rebar prices dip as mills lower offers in quiet market; HRC stable  
- 인도 HRC 밀 오퍼 경쟁력 있는 수준
- 인도 리바 입찰가 하락

Long Products  
- 아시아 빌렛 오퍼가 상승
- 아시아 시바운드 리바 가격 상승 
- 중국 국내 빌렛 가격 횡보

EMEA  
Flat Products
- 유럽 HRC 가격 하락 전망 부재로 보합 유지 예상  
- 9월까지 수요 부진 지속 전망
- 수입 HRC에 대한 관심 낮음

Long Products
- 터키 수출 리바 가격 박스권 등락
- 수출 시장 수요 약세
- 터키 리바 $577.5/mt FOB 평가(전일 대비 +$2.5)  

Raw Materials and Scrap
- 터키 국내 수입 스크랩 가격 보합 유지
- 단기 가격 전망 횡보
- 유럽 공급업체 오퍼 보합  

CIS
Long Products  
- 터키 빌렛 시장 경색, 중국산 오퍼가 바이어 타겟가와 근접
- 중국산 $515-518/mt CFR 터키 오퍼 vs 바이어 최고 $515/mt CFR 목표가

Americas
Industry news
- 중국 철강 시장 전망 부진, 경기부양책 기대에도 불구
- 부동산, 자동차 등 주요 수요처 수요 약세 지속 예상  
- 정부 경기부양책 효과는 제한적일 것으로 예상

- Tata Steel UK, 포트탤벗 제철소 5번 용광로 가동 중단
- 터키 5월 HR